This notebook shows how to get predictions from a model (same procedure for both trained or restored). In this notebook predictions from a restored model is used.

Note that you do not need this notebook to use modules. This notebook is just for displaying the usage of them. To make it easier to understand.<br>
<br><br>
Index of this notebook:<br>
1- Hyperparameters and initial setup <br>
2- Loading embedding model<br>
3- Restoring pre-trained model<br>
4- Getting predictions <br>


In [1]:
import helper
import EmbedHelper
import DataLoader
import Models

import tensorflow as tf
import numpy as np

C:\P\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


{'Dermatology': 'Deri ve Zührevi Hastalıkları (Cildiye)', 'Internal Medicine': 'İç Hastalıkları (Dahiliye)', 'Neurology': 'Nöroloji', 'Obstetrics & Gynecology': 'Kadın Hastalıkları ve Doğum', 'Ophthalmology': 'Göz Hastalıkları', 'Orthopaedic Surgery': 'Ortopedi ve Travmatoloji', 'Otolaryngology': 'Kulak Burun Boğaz Hastalıkları', 'Pediatrics': 'Çocuk Sağlığı ve Hastalıkları', 'Psychiatry': 'Ruh Sağlığı ve Hastalıkları', 'Radiology-Diagnostic': 'Radyoloji', 'Surgery-General': 'Genel Cerrahi', 'Urology': 'Üroloji'}


## 1- Hyperparameters and initial setup

In [2]:
embedDict = EmbedHelper.EmbeddingHandler.embedDict
print(embedDict)
configs = {
    "vectorSize":300,
    "trainNewModel":True,
    "dataColumn":"question",
    "maxLength":128,
    "batchSize":8,
    "embeddingType":embedDict[2],
    "PreEmbed":True,
    "restore":True,
    "model_type":"CNN_3Layer" # Options are : "CNN" (1 layer) , "CNN_3Layer", "RNN_LSTM"
}

inputSize = configs["maxLength"]
vectorSize = configs["vectorSize"]

{1: 'Fast Text', 2: 'Google News', 3: 'HealthTap', 4: 'Pubmed', 5: 'Glove', 6: 'iCliniq Trigram', 7: 'iCliniq default'}


## 2- Loading embedding model

In [3]:
# Load embedding model
EmbedModel = EmbedHelper.EmbeddingHandler(configs["embeddingType"], False, 300, "Embeddings")

Loading Google News


# 3- Restoring pre-trained model

In [4]:
# Remember that we used "test_model_name" when saving our model. Now we restore that model.
model_name = "test_model_name"

# Specify model path.
model_path = "NNModels/" + model_name + "/"+ configs["model_type"] + "model.ckpt"

# Output size 20 since we have 20 classes in data.
outputSize = 20

if configs["model_type"] == "CNN":
    nnModel = Models.CNN(inputSize=inputSize, vectorSize=vectorSize, outputSize=outputSize)
elif configs["model_type"] == "CNN_3Layer":
    nnModel = Models.CNN_3Layer(inputSize=inputSize, vectorSize=vectorSize, outputSize=outputSize)
elif configs["model_type"] == "RNN_LSTM":
    nnModel = Models.RNN_LSTM(inputSize=inputSize, vectorSize=vectorSize, outputSize=outputSize)

# Create session and assign model graph.
sess = tf.InteractiveSession(graph=nnModel.paperGraph)

# Initialize variables.
tf.global_variables_initializer().run()
sess.run(tf.tables_initializer())

# Restore the saved model.
tf.train.Saver().restore(sess, model_path)

fullvectorsize:  300
(?, 126, 1, 250)
Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Restoring parameters from NNModels/test_model_name/CNN_3Layermodel.ckpt


# 4- Getting Predictions

In [ ]:
# An example input, this can also be read from a file as text and put in array.
raw_user_input = ["Hello doctor, I have itchy skin problem."]

# Processing input so it can be fed into model.
user_input = DataLoader.DataHandler.cleanTextData(raw_user_input)
user_input = np.array(DataLoader.DataHandler.textIntoWordList(user_input, 128)[0])

# Getting the results which contains prediction.
results = helper.evaluatePerformance(configs, nnModel, EmbedModel, sess, user_input, [], 1, 0)

In [10]:
# We get our prediction from results.
prediction = helper.get_prediction_single_input(results)

In [11]:
print(prediction)

Internal Medicine


In [12]:
# overall accuracy if more than one data instance is evaluated
# suitable for testing accuracy over a dataset
# Would not work if there is no target data.
print(results["Accuracy"]) 

nan
